In [13]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timezone
from scipy.optimize import minimize, least_squares, differential_evolution
from scipy.integrate import quad

# --- Heston Pricing Functions ---
def heston_characteristic_function(phi, S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2):
    x = np.log(S)
    a = kappa * theta
    u = 0.5 if P1P2 == 1 else -0.5
    b = kappa - rho * volvol if P1P2 == 1 else kappa
    d = np.sqrt((rho * volvol * phi * 1j - b)**2 - volvol**2 * (2 * u * phi * 1j - phi**2))
    g = (b - rho * volvol * phi * 1j + d) / (b - rho * volvol * phi * 1j - d)
    C = (r - div) * phi * 1j * T + (a / volvol**2) * ((b - rho * volvol * phi * 1j + d) * T - 2 * np.log((1 - g * np.exp(d * T)) / (1 - g)))
    D = (b - rho * volvol * phi * 1j + d) / volvol**2 * (1 - np.exp(d * T)) / (1 - g * np.exp(d * T))
    return np.exp(C + D * var0 + 1j * phi * x)

def heston_call_price(S, K, T, r, kappa, rho, volvol, theta, var0, div):
    def integrand(phi, P1P2):
        cf = heston_characteristic_function(phi, S, K, T, r, kappa, rho, volvol, theta, var0, div, P1P2)
        return np.real(np.exp(-1j * phi * np.log(K)) * cf / (1j * phi))

    eps = 1e-6           # lower integration limit (avoids φ = 0)
    integral_P1 = quad(lambda phi: integrand(phi, 1),
                    eps, 100, limit=200, epsabs=1e-8, epsrel=1e-8)[0]
    integral_P2 = quad(lambda phi: integrand(phi, 2),
                    eps, 100, limit=200, epsabs=1e-8, epsrel=1e-8)[0]
    
    P1 = 0.5 + (1 / np.pi) * integral_P1
    P2 = 0.5 + (1 / np.pi) * integral_P2
    return max(0.0, S * np.exp(-div * T) * P1 - K * np.exp(-r * T) * P2)

def heston_put_price(S, K, T, r, kappa, rho, volvol, theta, var0, div):
    """Put option pricing function using put-call parity"""
    CallValue = heston_call_price(S, K, T, r, kappa, rho, volvol, theta, var0, div)
    PutValue = CallValue - S * np.exp(-div * T) + K * np.exp(-r * T)
    return PutValue

def heston_prices_parallel(params, Spots, Strikes, Maturities, Rates, div):
    kappa, rho, volvol, theta, var0 = params
    return np.array([
        heston_call_price(S, K, T, r, kappa, rho, volvol, theta, var0, div)
        for S, K, T, r in zip(Spots, Strikes, Maturities, Rates)
    ])

def mse_loss(Pmodel, Pmkt):
    return np.mean((Pmodel - Pmkt)**2)

def rmse_loss(Pmodel, Pmkt):
    return np.sqrt(np.mean((Pmodel - Pmkt)**2))

def relative_rmse_loss(Pmodel, Pmkt):
    return np.sqrt(np.mean(((Pmodel - Pmkt)/ np.mean(Pmkt))**2))

def OptFunction(params, Spots, Maturities, Rates, Strikes, MarketP, div, check_bounds=True):
    kappa, rho, volvol, theta, var0 = params
    if check_bounds and not (0.1 <= kappa <= 15 and -0.99 <= rho <= 0 and 0.01 <= volvol <= 2 and 0.001 <= theta <= 0.5 and 0.001 <= var0 <= 0.5):
        return 1e10

    mask = np.isfinite(MarketP) & (MarketP > 0)
    if not np.any(mask):
        return 1e10

    S, T, r, K, Pmkt = Spots[mask], Maturities[mask], Rates[mask], Strikes[mask], MarketP[mask]
    Pmodel = heston_prices_parallel(params, S, K, T, r, div)
    
    err = relative_rmse_loss(Pmodel, Pmkt)
    
    return err if np.isfinite(err) else 1e10


In [14]:
import yfinance as yf
from math import exp
import requests

FRED_API_KEY = ""
FRED_SERIES = {
    '1M': 'DGS1MO', '3M': 'DGS3MO', '6M': 'DGS6MO',
    '1Y': 'DGS1', '2Y': 'DGS2'
}

def fetch_fred_rates():
    rates = {}
    for label, sid in FRED_SERIES.items():
        url = f"https://api.stlouisfed.org/fred/series/observations?series_id={sid}&api_key=0de8b88e8310c6ebbd66c2eaa2ccb03f&file_type=json&sort_order=desc&limit=1"
        try:
            val = requests.get(url).json()['observations'][0]['value']
            if val != '.':
                rates[label] = float(val) / 100
        except:
            continue
    return rates

def get_rate_key(T):
    return (
        '1M' if T <= 1/12 else
        '3M' if T <= 0.25 else
        '6M' if T <= 0.5 else
        '1Y' if T <= 1 else
        '2Y'
    )

def get_option_calibration_data(symbol, target_expiration_str, max_main=20, max_side=15,nside=5):
    tk = yf.Ticker(symbol)
    expirations = pd.to_datetime(tk.options).date
    target_exp = pd.to_datetime(target_expiration_str).date()
    if target_exp not in expirations:
        raise ValueError("Target expiration not available.")
    
    idx = np.where(expirations == target_exp)[0][0]
    selected_dates = [expirations[i] for i in range(idx - nside-1, idx + nside) if 0 <= i < len(expirations)]

    spot = tk.history(period="1d")['Close'].iloc[-1]
    rates = fetch_fred_rates()
    today = datetime.today().date()
    data = []

    for expiry in selected_dates:
        try:
            df = tk.option_chain(expiry.isoformat()).calls
            df = df.dropna(subset=["bid", "ask", "impliedVolatility", "volume", "openInterest"])
            df = df[(df.bid > 0) & (df.ask > 0)].copy()
            df["midPrice"] = (df.bid + df.ask) / 2

            T = (expiry - today).days / 365
            rate = rates.get(get_rate_key(T), 0.0)
            F = spot * exp(rate * T)

            atm_strike = df.loc[(df.strike - F).abs().idxmin(), "strike"]
            df["moneyness"] = (df.strike - atm_strike).abs()
            n = max_main if expiry == target_exp else max_side

            selected = df.nsmallest(n, "moneyness").copy()
            selected["maturityDate"] = expiry
            selected["maturity"] = T
            selected["rate"] = rate
            selected["forward"] = F
            data.append(selected[[
                "maturityDate", "maturity", "strike", "midPrice",
                "impliedVolatility", "forward", "rate"
            ]])
        except:
            continue

    return pd.concat(data).reset_index(drop=True) if data else pd.DataFrame()

In [15]:
def save_calibration_results_to_excel(result, data, model_prices, file_path=None):
    """
    Save Heston calibration results to Excel including detailed option error analysis.
    
    Parameters:
    - result: The calibration result dictionary
    - data: DataFrame with market data
    - model_prices: Array of model prices
    - file_path: Path to save the Excel file (default: 'heston_calibration_results.xlsx')
    """
    if file_path is None:
        file_path = f'heston_calibration_{datetime.now().strftime("%Y%m%d_%H%M%S")}.xlsx'
    
    # Create a pandas Excel writer
    writer = pd.ExcelWriter(file_path, engine='xlsxwriter')
    
    # Create a detailed results dataframe
    detailed_df = data.copy()
    detailed_df['modelPrice'] = model_prices
    detailed_df['absoluteError'] = np.abs(detailed_df['modelPrice'] - detailed_df['midPrice'])
    detailed_df['relativeError'] = 100 * detailed_df['absoluteError'] / detailed_df['midPrice']
    detailed_df['squaredError'] = (detailed_df['modelPrice'] - detailed_df['midPrice'])**2
    
    # Save detailed results to Excel
    detailed_df.to_excel(writer, sheet_name='Option_Details', index=False)
    
    # Create summary by maturity
    maturity_summary = detailed_df.groupby('maturityDate').agg({
        'midPrice': ['mean', 'std', 'count'],
        'modelPrice': ['mean', 'std'],
        'absoluteError': ['mean', 'max', 'sum'],
        'relativeError': ['mean', 'max'],
        'squaredError': ['mean', 'sum']
    })
    maturity_summary.columns = ['_'.join(col).strip() for col in maturity_summary.columns.values]
    maturity_summary['rmse'] = np.sqrt(maturity_summary['squaredError_sum'] / maturity_summary['midPrice_count'])
    maturity_summary.to_excel(writer, sheet_name='Maturity_Summary')
    
    # Create summary of calibrated parameters
    params_df = pd.DataFrame([result['parameters']])
    params_df['rmse'] = rmse_loss(model_prices, detailed_df['midPrice'].values)
    params_df['relative_rmse'] = relative_rmse_loss(model_prices, detailed_df['midPrice'].values)
    params_df['optimization_time'] = result.get('optimization_time', result.get('total_time', 0))
    params_df['feller_condition'] = 2 * params_df['kappa'] * params_df['theta'] - params_df['volvol']**2
    params_df.to_excel(writer, sheet_name='Parameters', index=False)
    
    # Save and close
    writer.close()
    print(f"Calibration results saved to {file_path}")
    return file_path

In [ ]:
def Feller(x):
    kappa, rho, volvol, theta, var0 = x
    return 2 * kappa * theta - volvol**2

def calibrate_heston(symbol, expiration, div=0.0, save_excel=True, excel_path=None):
    t0 = time.time()
    data = get_option_calibration_data(symbol, expiration, max_main=5, max_side=5,nside=2)
    
    if data.empty:
        return {"success": False, "error": "No data found"}

    print(f"Data loaded in {time.time() - t0:.2f} seconds | {len(data)} options used")

    Spots      = data.forward.values
    Strikes    = data.strike.values
    Maturities = data.maturity.values
    Rates      = data.rate.values
    MarketP    = data.midPrice.values

    avg_iv = np.mean(data.impliedVolatility)
    var0 = avg_iv**2
    init = [1.5, -0.7, 0.6 * avg_iv, var0, var0]
    bounds = [(0.1, 10), (-0.95, 0.0), (0.01, 1.5), (0.001, 0.4), (0.001, 0.4)]
    cons = {'type': 'ineq', 'fun': Feller}

    t1 = time.time()
    # result = minimize(
    #     OptFunction, init, args=(Spots, Maturities, Rates, Strikes, MarketP, div, True),
    #     method="SLSQP", bounds=bounds, constraints=cons, options={"maxiter": 500, "disp": False}
    # )
    result = differential_evolution( OptFunction, bounds, args=(Spots, Maturities, Rates, Strikes, MarketP, div, False), maxiter=200, tol=1e-3)
    elapsed = (time.time() - t1) / 60

    xopt = result.x
    modelP = heston_prices_parallel(xopt, Spots, Strikes, Maturities, Rates, div)
    rmse = relative_rmse_loss(modelP, MarketP)

    print(result)
    print(f"Calibration time: {elapsed:.2f} min | RMSE: {rmse:.6f}")
    print(f"Feller condition: {Feller(xopt):.8f} > 0")
    print(f"Parameters: kappa={xopt[0]:.4f}, rho={xopt[1]:.4f}, volvol={xopt[2]:.4f}, theta={xopt[3]:.4f}, var0={xopt[4]:.4f}")

    # Save to Excel if requested
    if save_excel:
        if not excel_path:
            excel_path = f"{symbol}_heston_calibration.xlsx"
        save_calibration_results_to_excel(result, data, modelP, excel_path)
    
    return {
        "result": result,
        "success": result.success,
        "mse": rmse,
        "parameters": dict(zip(["kappa", "rho", "volvol", "theta", "var0"], xopt)),
        "optimization_time": elapsed,
        "total_time": time.time() - t0
    }

In [17]:
res = calibrate_heston(
    symbol='SPY',
    expiration='2025-07-31',
    div=0.0,
    save_excel=True,
    excel_path='differential_evolution_results.xlsx'
)

Data loaded in 4.57 seconds | 25 options used
             message: Optimization terminated successfully.
             success: True
                 fun: 0.009809161002426884
                   x: [ 9.934e+00 -4.289e-01  1.500e+00  2.866e-02
                        4.862e-02]
                 nit: 77
                nfev: 5958
          population: [[ 9.934e+00 -4.289e-01 ...  2.869e-02  4.857e-02]
                       [ 9.578e+00 -4.319e-01 ...  2.881e-02  4.831e-02]
                       ...
                       [ 9.986e+00 -4.283e-01 ...  2.866e-02  4.862e-02]
                       [ 9.871e+00 -4.223e-01 ...  2.864e-02  4.862e-02]]
 population_energies: [ 9.809e-03  9.838e-03 ...  9.821e-03  9.819e-03]
                 jac: [-3.528e-05 -1.467e-04 -9.460e-04  2.737e-04
                        3.472e-04]
Calibration time: 14.49 min | RMSE: 0.009809
Feller condition: -1.68053655 > 0
Parameters: kappa=9.9337, rho=-0.4289, volvol=1.5000, theta=0.0287, var0=0.0486


KeyError: 'parameters'

In [ ]:
import pandas as pd

# --- Calibration Function ---
def calibrate_heston(symbol, expiration, s0,
                                      div, max_options, max_time=300,
                                      methods=None,
                                      save_excel_path=None):
    if methods is None:
        methods = ["L-BFGS-B", "SLSQP", "LM"]

    start = time.time()
    
    data = get_clean_market_data(symbol, expiration, s0, max_options, min_maturity_days=5)
    
    print(f"Data loaded in {time.time() - start:.2f} seconds")
    print(f"Market data used: {len(data)} options")
    
    if data.empty:
        return {"success": False, "error": "No data found"}

    Spots    = data.forward.values
    Strikes  = data.strike.values
    Maturities = data.T.values
    Rates    = data.rate.values
    MarketP  = data.midPrice.values

    avg_iv   = np.mean(data.impliedVolatility)
    init_var = avg_iv ** 2
    initial_guess = [1.5, -0.7, 0.3 * avg_iv, init_var, init_var]
    bounds = [(0.1, 10.0), (-0.95, 0.0), (0.01, 1.5),
              (0.001, 0.4), (0.001, 0.4)]

    results = []
    detailed_results = []

    for method in methods:
        if time.time() - start > max_time:
            break
        try:
            t0 = time.time()
            if method == "LM":
                def residuals(p, Spots, Strikes, Mats, Rates, Market, div):
                    kappa, rho, volvol, theta, var0 = p
                    if not (0.1 <= kappa <= 15.0 and -0.99 <= rho <= 0.0
                            and 0.01 <= volvol <= 2.0 and 0.001 <= theta <= 0.5
                            and 0.001 <= var0 <= 0.5):
                        return np.full_like(Market, 1e5)
                    model = heston_prices_parallel(p, Spots, Strikes, Mats, Rates, div)
                    return model - Market

                res = least_squares(
                    residuals, initial_guess, method='lm',
                    args=(Spots, Strikes, Maturities, Rates, MarketP, div),
                    xtol=1e-12, ftol=1e-12, gtol=1e-12)
                xopt = res.x
                model_prices = heston_prices_parallel(
                    xopt, Spots, Strikes, Maturities, Rates, div)

            else:
                res = minimize(
                    OptFunctionFast, initial_guess, method=method, bounds=bounds,
                    args=(Spots, Maturities, Rates, Strikes, MarketP, div, True),
                    options={"maxiter": 500, "disp": False})
                xopt = res.x
                model_prices = heston_prices_parallel(
                    xopt, Spots, Strikes, Maturities, Rates, div)

            elapsed = time.time() - t0
            total_mse = np.mean((model_prices - MarketP) ** 2)
            per_option_mse = (model_prices - MarketP) ** 2

            # save full details
            detailed_results.append({
                "method": method,
                "total_mse": total_mse,
                "time": elapsed,
                "params": xopt,
                "model_prices": model_prices,
                "market_prices": MarketP,
                "per_option_mse": per_option_mse,
                "strikes": Strikes,
                "maturities": Maturities
            })

            results.append((method, total_mse, xopt, elapsed))
            print(f"Method: {method:11s} | Total MSE: {total_mse:.6f} | Time: {elapsed:.2f} s")

        except Exception as err:
            print(f"Method: {method:11s} failed → {err}")
            continue

    if not results:
        return {"success": False, "error": "All methods failed"}

    best_method, best_mse, best_params, best_time = min(results, key=lambda t: t[1])

    # --------- Save to Excel --------------
    if save_excel_path:
        writer = pd.ExcelWriter(save_excel_path, engine='xlsxwriter')

        for detail in detailed_results:
            df = pd.DataFrame({
                "Strike": detail["strikes"],
                "Maturity (years)": detail["maturities"],
                "Market Price": detail["market_prices"],
                "Model Price": detail["model_prices"],
                "Per Option MSE": detail["per_option_mse"],
                "Absolute Error": np.abs(detail["model_prices"] - detail["market_prices"]),
                "Relative Error (%)": 100 * np.abs(detail["model_prices"] - detail["market_prices"]) / detail["market_prices"]
            })

            sheet_name = detail["method"][:30]  # Excel max 31 chars
            df.to_excel(writer, sheet_name=sheet_name, index=False)

        # summary sheet
        summary = pd.DataFrame([{
            "Method": d["method"],
            "Total MSE": d["total_mse"],
            "Time (s)": d["time"],
            "Kappa": d["params"][0],
            "Rho": d["params"][1],
            "VolVol": d["params"][2],
            "Theta": d["params"][3],
            "V0": d["params"][4]
        } for d in detailed_results])
        summary.to_excel(writer, sheet_name="Summary", index=False)

        writer.close()  # Change from writer.save() to writer.close()
        print(f"Results saved to {save_excel_path}")

    # --------- Done --------------
    return {
        "success": True,
        "best_method": best_method,
        "mse": best_mse,
        "parameters": dict(zip(["kappa", "rho", "volvol", "theta", "var0"],
                               best_params)),
        "market_data_used": len(data),
        "calibration_time": time.time() - start,
        "all_results": [
            {"method": m, "mse": e, "params": p.tolist(), "time": t}
            for m, e, p, t in results
        ]
    }
